In [1]:
#libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Saved for later use
'''
all_data_sources.isnull().sum()

test_df = pd.DataFrame()
test_df["audio_features_track_id"] = lyrics_features["track_id"]
test_df["tracks_track_id"] = tracks["id"]

test_df['Match'] = test_df.apply(lambda x: 1 if x['audio_features_track_id'] in x['tracks_track_id'] else 0, axis=1)
'''

'\nall_data_sources.isnull().sum()\n\ntest_df = pd.DataFrame()\ntest_df["audio_features_track_id"] = lyrics_features["track_id"]\ntest_df["tracks_track_id"] = tracks["id"]\n\ntest_df[\'Match\'] = test_df.apply(lambda x: 1 if x[\'audio_features_track_id\'] in x[\'tracks_track_id\'] else 0, axis=1)\n'

In [2]:
albums = pd.read_csv('../../S2_Machine_Learning/SpotGenTrack/Data Sources/spotify_albums.csv', index_col = 0)
artists = pd.read_csv('../../S2_Machine_Learning/SpotGenTrack/Data Sources/spotify_artists.csv', index_col = 0)
tracks = pd.read_csv('../../S2_Machine_Learning/SpotGenTrack/Data Sources/spotify_tracks.csv', index_col = 0)
audio_features = pd.read_csv('../../S2_Machine_Learning/SpotGenTrack/Features Extracted/low_level_audio_features.csv', index_col = 0)
lyrics_features = pd.read_csv('../../S2_Machine_Learning/SpotGenTrack/Features Extracted/lyrics_features.csv', index_col = 0)

### Dimensionality Reduction

In [3]:
#Separate numeric and string columns (track_id) as PCA can only be performed on numeric columns
numeric_columns = audio_features.select_dtypes(include=['number']).columns
non_numeric_columns = audio_features.select_dtypes(exclude=['number']).columns

numeric_data = audio_features[numeric_columns]
non_numeric_data = audio_features[non_numeric_columns]

#Standardize numeric data
scaler = StandardScaler()
scaled_numeric_data = scaler.fit_transform(numeric_data)

#Redo PCA with number of components that explain 95% of the variance
pca = PCA(n_components=0.95)
principal_components = pca.fit_transform(scaled_numeric_data)
n_components = principal_components.shape[1]
principal_df = pd.DataFrame(data=principal_components, columns=[f'PC{i}' for i in range(1, n_components+1)])
#changed the name
processed_audio_features = pd.concat([non_numeric_data.reset_index(drop=True), principal_df], axis=1)

### Pre Processing

#### Tracks

In [4]:
#check for correlations of numerical features
num_attribs = tracks.select_dtypes(include=['float64', 'int64']).columns
numerical_corr_matrix = tracks[num_attribs].corr().abs()

# Display the correlations between the label (e.g., 'popularity') and other numerical columns
label_correlations = numerical_corr_matrix['popularity'].sort_values(ascending=False)
print(label_correlations)

popularity          1.000000
speechiness         0.304212
loudness            0.239679
duration_ms         0.169125
acousticness        0.154408
energy              0.153979
track_number        0.139460
time_signature      0.121505
danceability        0.092382
tempo               0.061869
liveness            0.059116
disc_number         0.047331
instrumentalness    0.041537
valence             0.034412
mode                0.010003
key                 0.000641
Name: popularity, dtype: float64


In [5]:
#include numerical columns with correlation greater than 0.09
include_col_tracks = ["playlist","available_markets","country","album_id","artists_id","id","lyrics", "time_signature","track_number","energy","acousticness","duration_ms","loudness","speechiness","danceability", "popularity"]
tracks_filtered = tracks[include_col_tracks]

#### Albums

In [6]:
albums_include = ['type','album_type', 'artist_id', 'available_markets','total_tracks', 'track_id', 'release_date',"id"]
albums_filtered = albums[albums_include]

#### Artists

In [7]:
artists_include = ['artist_popularity', 'followers', 'genres', 'id', 'name', 'track_id']
artists_filtered = artists[artists_include]

### Merging

In [8]:
#merge all dataframes
all_data_sources = tracks_filtered.merge(albums_filtered, how='left', left_on='album_id', right_on='id', suffixes=('_track','_albums')).merge(artists_filtered, how='left', left_on='artist_id', right_on='id', suffixes=('_track','_artists'))
all_data_sources = all_data_sources.merge(processed_audio_features, how="left", left_on="id_track", right_on='track_id', suffixes=('_track','_audio-features')).merge(lyrics_features, how="left", left_on="id_track", right_on='track_id', suffixes=('_tracks','_lyrics-features'))

In [9]:
all_data_sources.isnull().sum()

playlist                       0
available_markets_track        0
country                        0
album_id                       0
artists_id                     0
                            ... 
n_sentences                 6985
n_words                     6985
sentence_similarity         6985
track_id_lyrics-features    6985
vocabulary_wealth           6985
Length: 126, dtype: int64

In [10]:
all_data = all_data_sources
corr_num = all_data.corrwith(all_data['popularity']).abs().sort_values(ascending=False)
corr_num

/var/folders/vg/5yh2xcmn0msf83s5nl8f296r0000gn/T/ipykernel_9550/4040738530.py:2: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_num = all_data.corrwith(all_data['popularity']).abs().sort_values(ascending=False)


popularity           1.000000
artist_popularity    0.649121
speechiness          0.304212
followers            0.286363
loudness             0.239679
                       ...   
PC83                 0.001801
PC85                 0.001489
PC59                 0.001213
PC21                 0.001106
PC86                 0.000936
Length: 106, dtype: float64

In [11]:
all_data.columns.to_list()
drop_all = ["id_track","track_id_track","id_albums", "track_id_artists", "track_id_tracks", "track_id_lyrics-features", "album_id", "artists_id", "lyrics", "available_markets_albums", "available_markets_track"]  
all_data = all_data.drop(columns=drop_all)

In [12]:
all_data.columns.to_list()

['playlist',
 'country',
 'time_signature',
 'track_number',
 'energy',
 'acousticness',
 'duration_ms',
 'loudness',
 'speechiness',
 'danceability',
 'popularity',
 'type',
 'album_type',
 'artist_id',
 'total_tracks',
 'release_date',
 'artist_popularity',
 'followers',
 'genres',
 'id',
 'name',
 'PC1',
 'PC2',
 'PC3',
 'PC4',
 'PC5',
 'PC6',
 'PC7',
 'PC8',
 'PC9',
 'PC10',
 'PC11',
 'PC12',
 'PC13',
 'PC14',
 'PC15',
 'PC16',
 'PC17',
 'PC18',
 'PC19',
 'PC20',
 'PC21',
 'PC22',
 'PC23',
 'PC24',
 'PC25',
 'PC26',
 'PC27',
 'PC28',
 'PC29',
 'PC30',
 'PC31',
 'PC32',
 'PC33',
 'PC34',
 'PC35',
 'PC36',
 'PC37',
 'PC38',
 'PC39',
 'PC40',
 'PC41',
 'PC42',
 'PC43',
 'PC44',
 'PC45',
 'PC46',
 'PC47',
 'PC48',
 'PC49',
 'PC50',
 'PC51',
 'PC52',
 'PC53',
 'PC54',
 'PC55',
 'PC56',
 'PC57',
 'PC58',
 'PC59',
 'PC60',
 'PC61',
 'PC62',
 'PC63',
 'PC64',
 'PC65',
 'PC66',
 'PC67',
 'PC68',
 'PC69',
 'PC70',
 'PC71',
 'PC72',
 'PC73',
 'PC74',
 'PC75',
 'PC76',
 'PC77',
 'PC78',
 'PC79

### To Do
1. get Dataset with country names and population to calculate the potential listening population then drop available_markets_track. Same for available_markets_albums
2. Think about how the lyrics column could be used (top 3 word count? len of the lyrics?)

### Pre Pro Pipeline

In [13]:
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures

In [14]:
#define features and labels 
X = all_data.drop(columns=["popularity"])
y = all_data["popularity"]

In [15]:
train_data, test_data = train_test_split(all_data, test_size=0.2, random_state=42)
train_data.corr().sort_values(by="popularity",ascending=False)["popularity"]

/var/folders/vg/5yh2xcmn0msf83s5nl8f296r0000gn/T/ipykernel_9550/255386065.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train_data.corr().sort_values(by="popularity",ascending=False)["popularity"]


popularity           1.000000
artist_popularity    0.646580
followers            0.285169
loudness             0.237553
PC1                  0.203467
                       ...   
track_number        -0.134224
acousticness        -0.153994
duration_ms         -0.164460
PC4                 -0.208334
speechiness         -0.302021
Name: popularity, Length: 106, dtype: float64

In [16]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names]

    
def get_preprocessor(data):
    num_attribs = data.select_dtypes(include=['float64', 'int64']).columns
    cat_attribs = data.select_dtypes(include=['object']).columns
    
    num_pipeline = Pipeline([
            ('selector', DataFrameSelector(num_attribs)),
            ('imputer', SimpleImputer(strategy="median")),
            ('std_scaler', StandardScaler()),
        ])
    
    cat_pipeline = Pipeline([
            ('selector', DataFrameSelector(cat_attribs)),
            ('cat_encoder', OneHotEncoder(sparse=False)),
        ])
    
    full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])
    
    return full_pipeline

def create_prepared_dataframe(data, preprocessor, cat_attribs):
    # Get the transformed column names
    transformed_num_cols = preprocessor.transformers_[0][2].tolist()
    transformed_cat_cols = list(preprocessor.named_transformers_['cat']['cat_encoder'].get_feature_names_out(cat_attribs))
    
    # Combine the transformed column names
    transformed_cols = transformed_num_cols + transformed_cat_cols

    # Create a DataFrame with the transformed data and column names
    data_prepared = pd.DataFrame(data, columns=transformed_cols)

    return data_prepared

num_attribs = train_data.select_dtypes(include=['float64', 'int64']).columns
cat_attribs = train_data.select_dtypes(include=['object']).columns

preprocessor = get_preprocessor(train_data)
data_prepared_array = preprocessor.fit_transform(train_data[:500])

# Create a DataFrame with original column names
data_prepared_df = create_prepared_dataframe(data_prepared_array, preprocessor, cat_attribs)

/usr/local/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [17]:
data_prepared_df.corr()["popularity"].sort_values(ascending=False)

popularity             1.000000
artist_popularity      0.610101
followers              0.266291
loudness               0.232791
PC1                    0.223500
                         ...   
genres_['guidance']   -0.179179
total_tracks          -0.218432
speechiness           -0.272963
genres_[]             -0.303132
type_album                  NaN
Name: popularity, Length: 2704, dtype: float64

### Polynomial Features

In [18]:
#Create polynomial features
X = data_prepared_df.drop(columns="popularity")
y = data_prepared_df["popularity"]

degree = 2  #set to desired degree
poly_features = PolynomialFeatures(degree=degree)
X_poly = poly_features.fit_transform(X)

In [19]:
X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(X_poly, y, test_size=0.2, random_state=42)

In [21]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_poly, y_train_poly)

### Model

In [20]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint

X = data_prepared_df.drop(columns="popularity")
y = data_prepared_df["popularity"]

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42, error_score="raise")
rnd_search.fit(X, y)

RandomizedSearchCV(cv=5, error_score='raise',
                   estimator=RandomForestRegressor(random_state=42),
                   param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x13dd89310>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x13ab3f550>},
                   random_state=42, scoring='neg_mean_squared_error')

In [21]:
best_params = rnd_search.best_params_
print(f"Best parameters: {best_params}")

# Get the best mean squared error (MSE) score
best_mse = -rnd_search.best_score_
print(f"Best mean squared error: {best_mse}")

# Get the best estimator
best_estimator = rnd_search.best_estimator_

# Calculate R-squared using the best estimator
from sklearn.metrics import r2_score

y_pred = best_estimator.predict(X)
r2 = r2_score(y, y_pred)
print(f"R-squared: {r2}")

Best parameters: {'max_features': 7, 'n_estimators': 180}
Best mean squared error: 0.8323502441489197
R-squared: 0.8866781599780342


### Model Polynomial Features

In [22]:
forest_reg_poly = RandomForestRegressor(random_state=42)
rnd_search_poly = RandomizedSearchCV(forest_reg_poly, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42, error_score="raise")
rnd_search_poly.fit(X_poly, y)

best_params_poly = rnd_search_poly.best_params_
print(f"Best parameters: {best_params_poly}")

# Get the best mean squared error (MSE) score
best_mse_poly = -rnd_search_poly.best_score_
print(f"Best mean squared error: {best_mse_poly}")

# Get the best estimator
best_estimator_poly = rnd_search_poly.best_estimator_

# Calculate R-squared using the best estimator
from sklearn.metrics import r2_score

y_pred_poly = best_estimator_poly.predict(X_poly)
r2_poly = r2_score(y, y_pred_poly)
print(f"R-squared: {r2_poly}")

KeyboardInterrupt: 